In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [3]:
from d00_utils.db_utils import dbReadWriteViews, dbReadWriteMeasurement
io_views = dbReadWriteViews()
io_measurement = dbReadWriteMeasurement()

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
instances_w_labels_test_downsampleby5_df = io_views.get_table('instances_w_labels_test_downsampleby5')
instances_w_labels_test_downsampleby5_df = instances_w_labels_test_downsampleby5_df[(instances_w_labels_test_downsampleby5_df['view']=='a2c') | (instances_w_labels_test_downsampleby5_df['view']=='a4c')]
instanceidks = instances_w_labels_test_downsampleby5_df['instanceidk'].unique()
print(len(instanceidks))

522


In [5]:
from d05_measurement.write_groundtruth import write_groundtruth
write_groundtruth(instanceidks)

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
ground_truths_df = io_measurement.get_table('ground_truths')
print(ground_truths_df.shape)
ground_truths_df.head()

(704, 6)


,study_id,instance_id,file_name,measurement_name,measurement_value,measurement_unit
0,43763,779540,1LHFYKEZ,VTD(MDD-ps4),45.964351,mL
1,43763,779540,1LHFYKEZ,VTS(MDD-ps4),17.993407,mL
2,46919,841585,7SQZPLEJ,VTD(MDD-ps4),124.449981,mL
3,46919,841585,7SQZPLEJ,VTS(MDD-ps4),60.740148,mL
4,50521,921901,EAKXA40T,VTD(MDD-ps4),91.569690,mL


In [9]:
ground_truths_df.tail()

,study_id,instance_id,file_name,measurement_name,measurement_value,measurement_unit
699,126098,2985218,EBITNLTE,FE(MDD-ps4),64.646521,%
700,126288,2990141,EDTICRZG,FE(MDD-ps4),64.969289,%
701,126751,3003299,FDNHKCYK,FE(MDD-ps4),31.245536,%
702,126772,3003810,FDNHO33T,FE(MDD-ps4),74.126343,%
703,126781,3004026,FDNHQ1M9,FE(MDD-ps4),46.739345,%
